In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score



In [5]:
df = pd.read_csv('../comments/annotated.csv')

### Calculate cohens kappa

In [49]:
counts = df.groupby('commentId').agg(counts=('annotator', 'count')).reset_index()
df = df[df.commentId.isin(counts[counts.counts == 3].commentId)]

In [56]:
df = df.sort_values('commentId')

In [61]:
sentiment_map = {'Positive': 1, 'Neutral': 2, 'Negative': 3, 'Not applicable': 4}

In [65]:
ratings = np.zeros((int(df.shape[0] / 3), 3)) - 1
for annotator in [2, 3, 4]:
    mask = df.annotator == annotator
    ratings[:, annotator-2] = df[mask].sentiment.map(sentiment_map)

In [73]:
for i in range(3):
    for j in range(i, 3):
        if i != j:
            rater1 = ratings[:, i]
            rater2 = ratings[:, j]
            kappa = cohen_kappa_score(rater1, rater2)

            print(i, j, ':\t', kappa)

0 1 :	 0.7249698431845597
0 2 :	 0.7580645161290323
1 2 :	 0.7808619430241052


### Inspect cases of disagreement

In [150]:
mask = (ratings[:, 0] != ratings[:, 1]) & (ratings[:, 1] == ratings[:, 2]) & (ratings[:, 0] != ratings[:, 2])
df[['videoId', 'commentId', 'textOriginal', 'likeCount']].drop_duplicates()[mask].textOriginal

474    I remember after the 2nd Zombies was released,...
381    People who love Henry Cavill as Superman need ...
505    About time Lana Condor shows up in more movies...
257    "This is worse than that time Stewie suddenly ...
195                         Yeah, definitely not a cult.
396                                    Let's go Brandon!
59     "Is this the Regular Show?"\n"No, but some voi...
29                                #RestoreTheSnyderVerse
373    JG Quintel has reached his full power, no more...
357    Now wait for the parents to be like "ITS THE S...
Name: textOriginal, dtype: object

In [152]:
df[df.textOriginal.str.contains('who love Henry Cavill as Sup')]

,Unnamed: 0,videoId,commentId,textOriginal,likeCount,publishedAt,id,sentiment,annotator,annotation_id,created_at,updated_at,lead_time
381,25009,vM-Bja2Gy04,UgxBn1Xuhohwm6BrmTJ4AaABAg,People who love Henry Cavill as Superman need ...,3312,2021-02-27T03:10:37Z,61904,Neutral,2,1389,2022-12-12T15:47:39.322931Z,2022-12-12T15:47:39.322931Z,5.953
380,25009,vM-Bja2Gy04,UgxBn1Xuhohwm6BrmTJ4AaABAg,People who love Henry Cavill as Superman need ...,3312,2021-02-27T03:10:37Z,61904,Positive,3,1390,2022-12-12T15:47:40.368833Z,2022-12-12T15:47:40.368833Z,7.665
382,25009,vM-Bja2Gy04,UgxBn1Xuhohwm6BrmTJ4AaABAg,People who love Henry Cavill as Superman need ...,3312,2021-02-27T03:10:37Z,61904,Positive,4,1388,2022-12-12T15:47:36.754748Z,2022-12-12T15:47:36.754748Z,5.074


### Confusion matrix